## Deploy Models to the edge using SageMaker Edge Manager and IoT Greengrass v2

1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
3. [Setup](#Setup)
4. [Create an ARM64 instance](#Create-an-ARM64-instance)
5. [Prepare the Model for the Edge](#Prepare-the-Model-for-the-Edge)
    1. [Model](#Model)
    2. [Neo Compilation](#Neo-Compilation)
    3. [Package the model for Edge Manager](#Package-the-model-for-Edge-Manager)
6. [Setup Edge Device](#Setup-Edge-Device)
    1. [Install GreenGrass Core](#Install-GreenGrass-Core)
    2. [Create an Edge Manager device fleet](#Create-an-Edge-Manager-device-fleet)
    3. [Creat the device](#Creat-the-device)
7. [Build GreengrassV2 Component](#Build-GreengrassV2-Component)
8. [Deploy Using Greengrass](#Deploy-Using-Greengrass)
9. [SSH into the EC2 instance](#SSH-into-the-EC2-instance)
10. [Clean Up](#Clean-Up)

# Introduction

In this lab illustrate how you can optimize and deploy machine learning models for edge devices.

1. Use an ARM64 EC2 instance to mimic an typical ARM based edge device.
2. Config the edge device by installing Greengrass core to manage the edge deployment
3. Prepare our model for edge using SageMaker Neo and Edge Manager (either use the model from the previous modules or run the `optional-prepare-data-and-model.ipynb` notebook to create a new model)
4. Finally we will deploy a simple application that predicts a list of bird images and send the results to the cloud

** Note: This Notebook was tested on Data Science Kernel in SageMaker Studio**

# Prerequisites

Download the notebook into your environment, and you can run it by simply execute each cell in order. To understand what's happening, you'll need:

- Following Policy to your **SageMaker Studio Execution role** to properly executed this lab. <span style="color:red">Warning: the permissions set for this lab are very loose for simplicity purposes. Please follow the least privilege frame when you work on your own projects.  </span> These permissions allow user to interact with other AWS services like EC2, System Manager (SSM), IoT Core, and GreengrassV2.

    - AmazonEC2FullAccess
    - AmazonEC2RoleforSSM
    - AmazonSSMManagedInstanceCore
    - AmazonSSMFullAccess
    - AWSGreengrassFullAccess
    - AWSIoTFullAccess
    
- Familiarity with Python
- Basic understanding of SSM
- Basic familiarity with AWS S3
- Basic familiarity with IoT Core
- Basic familiarity with GreengrassV2
- Basic familiarity with AWS Command Line Interface (CLI) -- ideally, you should have it set up with credentials to access the AWS account you're running this notebook from.
- SageMaker Studio is preferred for the full UI integration

## Setup

Setting up the environment, load the libraries, and define the parameter for the entire notebook.

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = sess.default_bucket()
folder = "BIRD-Sagemaker-Edge"
compilation_output_sub_folder = folder + "/compilation-output"
iot_folder = folder + "/iot"

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = "s3://{}/{}".format(bucket, compilation_output_sub_folder)
s3_compilation_output_location

## Create an ARM64 instance

We will use an ARM64 EC2 instance to mimic the edge device.  If you do not have permission to create/update IAM roles, you can also complete the steps below using AWS Console. Please follow the [AWS Documentation](https://docs.aws.amazon.com/efs/latest/ug/gs-step-one-create-ec2-resources.html) to create an EC3 instance.

1. Select an `Arm 64 Ubuntu AMI`

<img src="static/1_select_AMI.png" width = 600 />

2. Select an `t4g.large instance type`

<img src="static/1_select_AMI.png" width = 600 />


3. Create an IAM role for your EC2 instance. follow this [AWS Documenation](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_for-service.html) to create a role for an AWS service.

3.1 Here is teh trust policy for the role
```
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "ec2.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
3.2 Add following policies

- CloudWatchAgentAdminPolicy
- AmazonS3FullAccess
- AmazonSSMManagedInstanceCore

4. Configure the security group to allow SSH from your IP address

5. Add a tag {"Key": "Name", "Value": "edge-manager-notebook"}

## If you use the console, Please Skip to [next section](#neo-compile-the-model) 


### Create EC2 Instance in Code

Again, skip this step if you created the instance using the console.

In [ ]:
ec2_client = boto3.client("ec2", region_name=region)
iam_client = boto3.client("iam", region_name=region)

### create the key pem file
Generate key pair for EC2 instance, save the key PEM file. We can use this key with SSH to connect to the instance. But in this notebook example, we will not use SSH, instead, we will use AWS Systems Manager to send commands to the instance.

In [ ]:
import datetime

ec2_key_name = "edge-manager-key-" + str(datetime.datetime.now())
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=ec2_key_name,
)

key_pair = str(ec2_key_pair["KeyMaterial"])
key_pair_file = open("ec2-key-pair.pem", "w")
key_pair_file.write(key_pair)
key_pair_file.close()

Get Ubuntu AMI Id

In [ ]:
ami = ec2_client.describe_images(Filters=[{'Name': 'name', 'Values': ['aws-parallelcluster-3.0.0b1-ubuntu-2004-lts-hvm-arm64-202108282001 2021-08-28T20-04-54.501Z']}])['Images'][0]['ImageId']
ami

### Create a role and allow EC2 instance to assume role using boto3

In [ ]:
ec2_trust_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "ec2.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}


role_name = f'Edge-Device-Role'

create_role_res = iam_client.create_role(
                                    RoleName=role_name,
                                    AssumeRolePolicyDocument=json.dumps(ec2_trust_policy),
                                    Description='This is role for ec2 instance to assume',
                                )

ec2_arn= create_role_res['Role']['Arn']

In [ ]:
create_inst_prof_res = iam_client.create_instance_profile(
        InstanceProfileName=role_name,
    )

create_inst_prof_res

In [ ]:
response = iam_client.add_role_to_instance_profile(
    InstanceProfileName=role_name,
    RoleName=role_name
)

In [ ]:
policy_list = [
    'arn:aws:iam::aws:policy/CloudWatchAgentAdminPolicy',
     'arn:aws:iam::aws:policy/AmazonS3FullAccess',
     'arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore',
    'arn:aws:iam::aws:policy/AWSIoTFullAccess',
    'arn:aws:iam::aws:policy/IAMFullAccess'
    
]

for p in policy_list:
    policy_attach_res = iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn=p
    )
        
    print(policy_attach_res)

### Create the t4g large instance

T4g instances are powered by Arm-based AWS Graviton2 processors. [Here](https://aws.amazon.com/ec2/instance-types/t4/) about this instance type.  We are going to use it to mimic an Arm64 edge device.

Previous step make take a few minutes to complete, please retry the code block below if yuo get an error about InvalidParameterValue.  While the instance is spinning up, we can go to the model preparation steps next.

In [ ]:
ec2_instance = ec2_client.run_instances(
    ImageId=ami,
    MinCount=1,
    MaxCount=1,
    InstanceType="t4g.large",
    KeyName=ec2_key_name,
    IamInstanceProfile={
        "Name": role_name,
    },
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "edge-manager-notebook"}],
        }
    ],
)

instance_id = ec2_instance["Instances"][0]["InstanceId"]
print(f"Instance ID is {instance_id}")

## Prepare the Model for the Edge

### Model

You can use the bird model use created from the previous modules or run the `optional-prepare-data-and-model.ipynb` notebook to create a new model. Update the path to your model below if necessary.

In [ ]:
bird_model_path = '<S3 LOCATION OF YOUR MODEL ARTIFACTS>'

### Neo Compilation
Amazon SageMaker Neo enables developers to optimize machine learning (ML) models for inference on SageMaker in the cloud and supported devices at the edge.

Amazon SageMaker Neo automatically optimizes machine learning models for inference on cloud instances and edge devices to run faster with no loss in accuracy. You start with a machine learning model trained in Amazon SageMaker or anywhere else. Then you choose your target hardware platform, which can be a SageMaker hosting instance or an edge device based on processors from Ambarella, Apple, ARM, Intel, MediaTek, Nvidia, NXP, Qualcomm, RockChip, Texas Instruments, or Xilinx.

In this example we will be using our Bird model from previous lab.

We are compiling a MobileNetV2 model in TensorFlow.  The input tensor shape is `"input_1":[1,224,224,3]`, The edge device is our t4g instance which is a ARM64 device running Linux.

In [ ]:
sagemaker_client = boto3.client("sagemaker", region_name=region)

s3_client = boto3.client("s3", region_name=region)

The TF2 model is already provided with the lab.  We need to upload the model to S3 first.

In [ ]:
bird_data_shape = '{"input_1":[1,224,224,3]}'
bird_framework = "TensorFlow"

target_os = "LINUX"
target_arch = "ARM64"

In [ ]:
import time

bird_compilation_job_name = "Sagemaker-Bird-" + str(time.time()).split(".")[0]
print("Compilation job for %s started" % bird_compilation_job_name)

response = sagemaker_client.create_compilation_job(
    CompilationJobName=bird_compilation_job_name,
    RoleArn=role,
    InputConfig={
        "S3Uri": bird_model_path,
        "DataInputConfig": bird_data_shape,
        "Framework": bird_framework.upper(),
    },
    OutputConfig={
        "S3OutputLocation": s3_compilation_output_location,
        "TargetPlatform": {"Arch": target_arch, "Os": target_os},
    },
    StoppingCondition={"MaxRuntimeInSeconds": 900},
)

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(
        CompilationJobName=bird_compilation_job_name
    )
    if response["CompilationJobStatus"] == "COMPLETED":
        break
    elif response["CompilationJobStatus"] == "FAILED":
        print(response)
        raise RuntimeError("Compilation failed")
    print("Compiling ...")
    time.sleep(30)
print("Done!")

### Package the model for Edge Manager

Once the model is neo compiled, we will use Edge Manager to re-package the model and create the component for Greengrass deployment.

In [ ]:
!pip install boto3 --upgrade

In [ ]:
bird_packaged_model_name = "bird-model"
bird_model_version = "1.0"
bird_model_package = "{}-{}.tar.gz".format(bird_packaged_model_name, bird_model_version)
model_component_version = "1.0.0"
model_component_name = 'gg-bird-component'

In [ ]:
bird_packaging_job_name = bird_compilation_job_name + "-packaging1"

response = sagemaker_client.create_edge_packaging_job(
    EdgePackagingJobName=bird_packaging_job_name,
    CompilationJobName=bird_compilation_job_name,
    ModelName=bird_packaged_model_name,
    ModelVersion=bird_model_version,
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': f"s3://{bucket}/artifacts/{model_component_name}/{model_component_version}/",
        "PresetDeploymentType":"GreengrassV2Component",
        "PresetDeploymentConfig":'{"ComponentName":"'+ model_component_name +'", "ComponentVersion":"'+ model_component_version +'"}'
    }

)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(
        EdgePackagingJobName=bird_packaging_job_name
    )
    if job_status["EdgePackagingJobStatus"] == "COMPLETED":
        break
    elif job_status["EdgePackagingJobStatus"] == "FAILED":
        print(job_status)
        raise RuntimeError("Edge Packaging failed")
    print("Packaging ...")
    time.sleep(30)
    
bird_model_data = job_status["ModelArtifact"]

print(f'Package data is located here: {bird_model_data}')

print("Done!")

In [ ]:
ssm_client = boto3.client("ssm", region_name=region)

In [ ]:
iot_thing_name = "bird-edge-thing-"+ str(time.time()).split(".")[0]
iot_thing_group = f"{iot_thing_name}-group"
tes_role = f'SageMaker-TES-Role-{str(time.time()).split(".")[0]}'
tes_alias = f'{tes_role}-Alias'

print(f'Thing group name: {iot_thing_group}')
print(f'TES Alias: {tes_alias}')

run_command = 'sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ./GreengrassInstaller/lib/Greengrass.jar ' 
run_command += f'--aws-region {region} --thing-name {iot_thing_name} --thing-group-name {iot_thing_group} '
run_command += f'--tes-role-name {tes_role} --tes-role-alias-name {tes_alias} '
run_command += '--component-default-user ggc_user:ggc_group --provision true --setup-system-service true'

response = ssm_client.send_command(
    InstanceIds=[instance_id],
    DocumentName="AWS-RunShellScript",
    OutputS3BucketName=bucket,
    OutputS3KeyPrefix=folder,
    Parameters={
        "commands": [
            "#!/bin/bash",
            "wget -O- https://apt.corretto.aws/corretto.key | sudo apt-key add -",
            "sudo add-apt-repository 'deb https://apt.corretto.aws stable main'",
            "sudo apt-get update; sudo apt-get install -y java-1.8.0-amazon-corretto-jdk",
            "sudo apt-get update && sudo apt-get install python3-venv -y",
            "curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip",
            "unzip greengrass-nucleus-latest.zip -d GreengrassInstaller && rm greengrass-nucleus-latest.zip",
            run_command
        ]
    },
)

Check the status of your command, they may take a few minutes.

In [ ]:
ssm_client.get_command_invocation(
    CommandId=response["Command"]["CommandId"],
    InstanceId=instance_id,
)

### Create an Edge Manager device fleet

Create an Edge Manager fleet to manage multiple devices at once.  There is a per account limit of 10 fleet. If you need go beyond 10, please contact your account team to increase the limit.

In addition to creating the fleet, you also need to configure an IAM role that will be assumed by the credentials provider on behalf of the devices in your device fleet. 

**Notice**: The name of the role must start with `SageMaker`.

Go to [IAM console](https://console.aws.amazon.com/iam), create role for IoT, attach the following policies:

- AmazonSageMakerEdgeDeviceFleetPolicy

In [ ]:
iot_trust_policy = {
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}

update_role_res = iam_client.update_assume_role_policy(
                                    RoleName=tes_role,
                                    PolicyDocument=json.dumps(iot_trust_policy),
                                )


In [ ]:
policy_list = [
    'arn:aws:iam::aws:policy/service-role/AmazonSageMakerEdgeDeviceFleetPolicy',
    'arn:aws:iam::aws:policy/AmazonS3FullAccess'
]

for p in policy_list:
    policy_attach_res = iam_client.attach_role_policy(
        RoleName=tes_role,
        PolicyArn=p
    )
        
    print(policy_attach_res)

In [ ]:
print(tes_role)
role_resp = iam_client.get_role(
    RoleName=tes_role,
)

fleet_arn = role_resp['Role']['Arn']

fleet_arn

It may take a second or two to generate a role, so if you get an role doesn't exist error, just run the code below again.

In [ ]:
device_fleet_name = "bird-classification-device-fleet-" + str(time.time()).split(".")[0]

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=fleet_arn,
    OutputConfig={"S3OutputLocation": s3_compilation_output_location},
)

print(f'Device Fleet Name: {device_fleet_name}')

### Creat the device
associate the device to the IOT Core and the fleet

In [ ]:
device_name = (
    "sagemaker-edge-demo-device" + str(time.time()).split(".")[0]
)  # device name should be 36 charactors

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device",
        }
    ],
)

## Build GreengrassV2 Component

AWS IoT Greengrass components are software modules that you deploy to Greengrass core devices. Components can represent applications, runtime installers, libraries, or any code that you would run on a device. You can define components that depend on other components. For example, you might define a component that installs Python, and then define that component as a dependency of your components that run Python applications. When you deploy your components to your fleets of devices, Greengrass deploys only the software modules that your devices require.

There are the compnents for this lab:
- Bird-Model-Test-Script2
- Bird-Model-ARM-TF2
- aws.greengrass.SageMakerEdgeManager (public component)

In [ ]:
import os

def upload_artifacts(file_name, object_name=None):
    if object_name is None:
        object_name = file_name
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        
    except Exception as e:
        print(
            "Failed to upload the artifacts to the bucket {} with key {}.\nException: {}".format(
                bucket, object_name, e
            )
        )
        exit(1)
    
    
    print(
        "Successfully uploaded the artifacts to the bucket {} with key {}.".format(
            bucket, object_name
        )
    )
    return f"s3://{bucket}/{object_name}"

**Upload component artifacts files to S3**

Components can have any number of artifacts, which are component binaries. Artifacts can include scripts, compiled code, static resources, and any other files that a component consumes. Components can also consume artifacts from component dependencies.

In [ ]:
import shutil

# zip the script files in build folder.
shutil.make_archive('build/image_classification', 'zip', 'build/image_classification')

script_component_name = "Bird-Model-Test-Script2"
script_component_version = "1.0.1"
model_component_path = f"artifacts/{script_component_name}/{script_component_name}"


s3_path = dict()

for file in os.listdir('build'):

    file_path = os.path.join('build', file)
    object_name = f"{model_component_path}/{file}"
    
    if ".zip" in file_path:
        s3_path['code'] = upload_artifacts(file_path, object_name)
    if ".sh" in file_path:
        s3_path['build'] = upload_artifacts(file_path, object_name)
    else:
        pass
        
s3_path

**Build the component Recipe**

Every component contains a recipe file, which defines its metadata. The recipe also specifies the component's configuration parameters, component dependencies, lifecycle, and platform compatibility. The component lifecycle defines the commands that install, run, and shut down the component. For more information, see [AWS IoT Greengrass component recipe reference.](https://docs.aws.amazon.com/greengrass/v2/developerguide/component-recipe-reference.html)

You can define recipes in JSON or YAML format.


In [ ]:
import json

application_recipe = {
    "RecipeFormatVersion": "2020-01-25",
    "ComponentName": script_component_name,
    "ComponentVersion": script_component_version,
    "ComponentDescription": "Sagemaker edge manager python example of image classification deployed using Greengrassv2.",
    "ComponentPublisher": "AWS",
    "ComponentConfiguration": {
        "DefaultConfiguration": {
            "accessControl": {
                "aws.greengrass.ipc.mqttproxy": {
                    "com.greengrass.SageMakerEdgeManager.ImageClassification:mqttproxy:1": {
                        "policyDescription": "Allows access to publish via topic gg/sageMakerEdgeManager/image-classification.",
                        "operations": [
                            "aws.greengrass#PublishToIoTCore"
                        ],
                        "resources": [
                            "gg/sageMakerEdgeManager/image-classification"
                        ]
                    }
                }
            },
            "ImageName": "Cardinal_0001_17057.jpg",
            "InferenceInterval": "3600",
            "PublishResultsOnTopic": "gg/sageMakerEdgeManager/image-classification"
        }
    },
    "Manifests": [
        {
            "Platform": {
                "os": "linux",
                "architecture": "aarch64"
            },
            "Lifecycle": {                
                "setEnv": {
                    "DEFAULT_SMEM_IC_IMAGE_DIR": "{artifacts:decompressedPath}/image_classification/image_classification/images/",
                },
                "run": {
                    "script": "bash {artifacts:path}/installer.sh\npython3 {artifacts:decompressedPath}/image_classification/image_classification/inference.py"
                }
            },
            "Artifacts": [
                {
                    "Uri": s3_path['code'],
                    "Unarchive": "ZIP",
                    "Permission": {
                        "Read": "OWNER",
                        "Execute": "NONE"
                    }
                },
                {
                    "Uri": s3_path['build'],
                    "Unarchive": "NONE",
                    "Permission": {
                        "Read": "OWNER",
                        "Execute": "NONE"
                    }
                }
            ]
        }
    ]
}


greengrass_client = boto3.client("greengrassv2", region_name=region)
greengrass_resp = greengrass_client.create_component_version(inlineRecipe=json.dumps(application_recipe))

greengrass_resp

## Deploy Using Greengrass

To deploy your components (console)

1. In the [AWS IoT Greengrass console navigation menu](https://console.aws.amazon.com/greengrass), choose Deployments, and then choose the deployment for your target device that you want to revise.

2. On the deployment page, choose **Create**.  If you are updating an existing deployment then choose Revise and then choose Revise deployment.

3. On the Specify target page, select the target IoT group and choose Next.

4. On the Select components page, do the following:

   - Under My components, select the following components:

   - Bird-Model-Test-Script2

   - Bird-Model-ARM-TF2

   - Under Public components, turn off the Show only selected components toggle, and then select the aws.greengrass.SageMakerEdgeManager component.

   - Choose Next.

5. On the Configure components page, select the aws.greengrass.SageMakerEdgeManager component and do the following.

   - Choose Configure component.

   - Under Configuration update, in Configuration to merge, enter the following configuration.
```
{
    "DeviceFleetName": "device-fleet-name",
    "BucketName": "BUCKET-Start-With-Sagemaker",
    "FolderPrefix": "BIRD-Sagemaker-Edge"
}
```
   - Replace device-fleet-name with the name of the edge device fleet that you created, and replace DOC-EXAMPLE-BUCKET with the name of the S3 bucket that is associated with your device fleet.

   - Choose Confirm, and then choose Next.

6. On the Configure advanced settings page, keep the default configuration settings, and choose Next.

7. On the Review page, choose Deploy

<img src="static/4_deploy_greengrass.png" width = 900 />

### Deploy Using Code

In [ ]:
iot_client = boto3.client('iot')
response = iot_client.describe_thing_group(
    thingGroupName=iot_thing_group
)

target_arn = response['thingGroupArn']
target_arn

In [ ]:
response = greengrass_client.create_deployment(
    targetArn=target_arn,
    deploymentName="bird-edge-demo2",
    components={
        script_component_name: {
            "componentVersion": script_component_version,
        },
        model_component_name:{
            "componentVersion":model_component_version,
        },
        "aws.greengrass.SageMakerEdgeManager":{
            "componentVersion":"1.0.3",
            'configurationUpdate': {
                'merge': json.dumps({
                    "DeviceFleetName": device_fleet_name,
                    "BucketName": bucket,
                    "FolderPrefix":folder
                })
            },
            
        }
    },
)
response

## View Your Deployment in GreenGrass Console

1. Go to [Greengrass Console](https://console.aws.amazon.com/greengrass/)

2. View the deployment you just made in the deployment dashoboard
   
   - This shows you the deployment details: components, target devices, and deployment status
   - You can also see the device status to indicate the device is running in a healthy state.

<img src="static/5_deployment_dashboard.png" width = 900 />

3. Click inot Core device to get more detail on the device status
   - You can get status details on each components

<img src="static/6_monitoring_core_device.png" width = 900 />

## SSH into the EC2 instance

In addition to monitor the device in the console, you can also SSH and see how different component interact with each other.  We recommend to use AWS instance connect to SSH into the device.  [Here](https://www.youtube.com/watch?v=lxSNeF7BAII&ab_channel=StephaneMaarek) is a nice quick tutorial on how to do that.

Basically you need to open up SSH inbound rule in your security group.  To use instance connect the security group on the EC2 instance should allow inbound connections from the EC2 Instance Connect service, not your own IP address.

You can obtain the relevant IP address range from [AWS IP Address Ranges](https://docs.aws.amazon.com/general/latest/gr/aws-ip-ranges.html).

For example, here is the range for the Sydney region:
```
{
    "ip_prefix": "13.239.158.0/29",
    "region": "ap-southeast-2",
    "service": "EC2_INSTANCE_CONNECT"
},
```

Your components deployed on the device will be located at `/greengrass/v2/packages/` directory.

All the logs will be located at `/greengrass/v2/logs/`.  There are 3 logs that are important to monitor:

- Script Component Log: `/greengrass/v2/logs/Bird-Model-Test-Script2.log`
- Greengrass log: `/greengrass/v2/logs/greengrass.log`
- Edge Manager log: `/greengrass/v2/logs/aws.greengrass.SageMakerEdgeManager.log`

All these directory needs admin user permission to access, so the easiest way is to `sudo su` when you SSH onto the device.

## Clean Up
Don't Forget to clean up the resource you created after you are done.